## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-09-23-01-20 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/02/08/world/...
1,2026-02-09-22-58-00 +0000,wsj,Fed’s Miran Says Dollar Needs ‘Really Big Move...,https://www.wsj.com/economy/central-banking/fe...
2,2026-02-09-22-54-13 +0000,nypost,"Eileen Gu, American-born Olympic star who comp...",https://nypost.com/2026/02/09/us-news/eileen-g...
3,2026-02-09-22-50-15 +0000,cbc,Liberal MP Bruce Fanjoy criticizes government'...,https://www.cbc.ca/news/canada/ottawa/bruce-fa...
4,2026-02-09-22-49-00 +0000,wsj,Opinion | The Political Battle for AI in Space,https://www.wsj.com/opinion/elon-musk-xai-spac...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2451/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
272,super,27
273,bowl,27
19,trump,19
268,bad,17
274,halftime,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
290,2026-02-09-02-55-54 +0000,nypost,Trump slams Bad Bunny’s Super Bowl halftime sh...,https://nypost.com/2026/02/08/us-news/trump-sl...,144
195,2026-02-09-13-23-19 +0000,nyt,Trump Criticizes Bad Bunny’s Super Bowl Halfti...,https://www.nytimes.com/2026/02/09/us/trump-ba...,139
172,2026-02-09-15-34-43 +0000,nyt,The Wedding During Bad Bunny’s Super Bowl Half...,https://www.nytimes.com/2026/02/09/us/super-bo...,128
244,2026-02-09-09-36-38 +0000,bbc,Bad Bunny makes history as Trump criticises 't...,https://www.bbc.com/news/articles/c394g7nnzmzo...,127
286,2026-02-09-03-08-19 +0000,nyt,"Lady Gaga, Ricky Martin and Cardi B Join Bad B...",https://www.nytimes.com/2026/02/08/us/bad-bunn...,125


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
290,144,2026-02-09-02-55-54 +0000,nypost,Trump slams Bad Bunny’s Super Bowl halftime sh...,https://nypost.com/2026/02/08/us-news/trump-sl...
70,64,2026-02-09-20-54-32 +0000,nypost,Savannah Guthrie posts new video begging publi...,https://nypost.com/2026/02/09/us-news/savannah...
245,61,2026-02-09-09-11-41 +0000,nypost,China critic and former media tycoon Jimmy Lai...,https://nypost.com/2026/02/09/world-news/china...
165,53,2026-02-09-16-00-19 +0000,cbc,Air Canada halts Cuba flights as Havana airpor...,https://www.cbc.ca/news/canada/travel-to-cuba-...
92,50,2026-02-09-19-53-00 +0000,wsj,U.K.’s Starmer Is in Danger of Becoming First ...,https://www.wsj.com/world/uk/u-k-s-starmer-is-...
309,41,2026-02-09-00-57-00 +0000,nypost,Japanese PM Sanae Takaichi’s party sweeps to m...,https://nypost.com/2026/02/08/world-news/japan...
26,39,2026-02-09-22-14-00 +0000,wsj,The White House is at loggerheads with Congres...,https://www.wsj.com/politics/policy/proposed-b...
32,37,2026-02-09-22-08-06 +0000,nyt,How Ilia Malinin and Team U.S.A. Held Off Japa...,https://www.nytimes.com/2026/02/08/world/how-t...
69,33,2026-02-09-20-58-59 +0000,nyt,"San Francisco Teachers Begin Strike, Closing S...",https://www.nytimes.com/2026/02/09/us/san-fran...
95,30,2026-02-09-19-47-58 +0000,cbc,Lindsey Vonn says she'll need multiple surgeri...,https://www.cbc.ca/sports/olympics/winter/alpi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
